In [3]:
import os

ZIP_PATH = "/content/faces.zip"
print(os.path.exists(ZIP_PATH))
print(os.path.getsize(ZIP_PATH) if os.path.exists(ZIP_PATH) else "No file")


True
126877696


In [5]:
!file /content/faces.zip


/content/faces.zip: Zip archive data, at least v4.5 to extract, compression method=deflate


In [6]:
!unzip /content/faces.zip -d /content/data


Streaming output truncated to the last 5000 lines.
  inflating: /content/data/Face Data/Face Dataset/1618/0.jpg  
  inflating: /content/data/Face Data/Face Dataset/1618/1.jpg  
  inflating: /content/data/Face Data/Face Dataset/1618/2.jpg  
  inflating: /content/data/Face Data/Face Dataset/1619/0.jpg  
  inflating: /content/data/Face Data/Face Dataset/1619/1.jpg  
  inflating: /content/data/Face Data/Face Dataset/1619/2.jpg  
  inflating: /content/data/Face Data/Face Dataset/1619/3.jpg  
  inflating: /content/data/Face Data/Face Dataset/1619/4.jpg  
  inflating: /content/data/Face Data/Face Dataset/162/0.jpg  
  inflating: /content/data/Face Data/Face Dataset/162/1.jpg  
  inflating: /content/data/Face Data/Face Dataset/162/2.jpg  
  inflating: /content/data/Face Data/Face Dataset/1620/0.jpg  
  inflating: /content/data/Face Data/Face Dataset/1620/1.jpg  
  inflating: /content/data/Face Data/Face Dataset/1621/0.jpg  
  inflating: /content/data/Face Data/Face Dataset/1621/1.jpg  
  infla

In [8]:
DATASET_DIR = "/content/data/Face Data/Face Dataset"


In [9]:
!ls "/content/data/Face Data/Face Dataset" | head


0
1
10
100
1000
1001
1002
1003
1004
1005


In [10]:
DATASET_DIR = "/content/data/Face Data/Face Dataset"


In [12]:
import os

def load_identities(base_dir):
    identities = {}
    for pid in os.listdir(base_dir):
        pdir = os.path.join(base_dir, pid)
        imgs = [os.path.join(pdir, f) for f in os.listdir(pdir)]
        if len(imgs) >= 2:
            identities[pid] = imgs
    return identities


In [13]:
DATASET_DIR = "/content/data/Face Data/Face Dataset"
identities = load_identities(DATASET_DIR)
print("Valid identities:", len(identities))


Valid identities: 1680


In [14]:
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 20


In [15]:
import cv2
import numpy as np

def preprocess_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img / 255.0


In [29]:
def triplet_generator(identities, batch_size):
    ids = list(identities.keys())

    while True:
        A, P, N = [], [], []

        for _ in range(batch_size):
            pid = random.choice(ids)
            a, p = random.sample(identities[pid], 2)

            neg_pid = random.choice([x for x in ids if x != pid])
            n = random.choice(identities[neg_pid])

            A.append(preprocess_image(a))
            P.append(preprocess_image(p))
            N.append(preprocess_image(n))

        yield (
            (np.array(A), np.array(P), np.array(N)),
            np.zeros((batch_size,))
        )




In [30]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_embedding():
    model = models.Sequential([
        layers.Input((IMG_SIZE, IMG_SIZE, 3)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation="relu"),
        layers.GlobalAveragePooling2D(),
        layers.Dense(128),
        layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ])
    return model

embedding_model = build_embedding()
embedding_model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_2 (Lambda)               │ (None, 128)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,760 (428.75 KB)

 Trainable params: 109,760 (428.75 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
class SiameseModel(tf.keras.Model):
    def __init__(self, embedding, margin=0.2):
        super().__init__()
        self.embedding = embedding
        self.margin = margin

    def train_step(self, data):
        (A, P, N), _ = data

        with tf.GradientTape() as tape:
            a = self.embedding(A, training=True)
            p = self.embedding(P, training=True)
            n = self.embedding(N, training=True)

            pos = tf.reduce_sum(tf.square(a - p), axis=1)
            neg = tf.reduce_sum(tf.square(a - n), axis=1)
            loss = tf.reduce_mean(tf.maximum(pos - neg + self.margin, 0))

        grads = tape.gradient(loss, self.embedding.trainable_variables)
        self.optimizer.apply_gradients(
            zip(grads, self.embedding.trainable_variables)
        )

        return {"loss": loss}


In [36]:
siamese = SiameseModel(embedding_model)

siamese.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4)
)


In [37]:
history = siamese.fit(
    triplet_generator(identities, BATCH_SIZE),
    steps_per_epoch=len(identities)//BATCH_SIZE,
    epochs=EPOCHS
)


Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - loss: 0.1899
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - loss: 0.1755
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - loss: 0.1721
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - loss: 0.1726
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 0.1720
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - loss: 0.1645
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.1634
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 0.1626
Epoch 9/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - loss: 0.1515
Epoch 10/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 0.1586
Epoch 11/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - loss: 0.1516
Epoch 12/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.1600
Epoch 13/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 53ms/step - loss: 0.1585
Epoch 14/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - loss: 0.1463
Epoch 15/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 5

In [39]:
embedding_model.save("siamese_model.keras")
print("Model saved successfully")



Model saved successfully
